# Generate chips aligned with Sentinel 2 pixels

Steps:

* Read point coordinates in GeoDataFrame.
* Run `geoimgchips.s2_raster_aligned_chips_from_points` with GeoDataFrame and a buffer in [m]. Steps:
  * Derive UTM zone, EPSG number and coordinates from lat/lon coordinates
    ([Sentinel-2 UTM Tiling Grid (ESA)](https://eatlas.org.au/data/uuid/f7468d15-12be-4e3f-a246-b2882a324f59))
  * Create polygons around the points with the given buffer
  * Grow polygon to aligned with closest 10m, 20m or 60m Sentinel-2 pixel borders
  * Return GeoDataFrame witch chip polygons and utm information
* Write chips to file.

## Imports

In [1]:
import geoimgchips
import geopandas as gpd
from pathlib import Path

## Parameters

In [2]:
# geopandas.read_file(src_file) readable file with lat/long points
# here we use a test dataset comimg with the geoimgchips package
src_file = geoimgchips.datasets.get_path(dataset="five_points.geojson")

# destination file for chips
dst_file_polygons = "./uncommitted_data/chips.geojson"

# size of the chip
# e.g. 1280 => 256 x 256 10m pixels, size can differ if res != 10
buffer = (2**8) * 10 / 2

# aligns chip borders for a given pixel resolution
res = 10

# subset of columns from the source file to keep for the destination files
# default `None` will keep all
# if given, geometry needs to be included
# e.g. ['id', 'tile', 'geometry']
src_keep_columns = None

# overwrite destination files if they exist
overwrite = True

## Processing

Check if something needs to be done.

In [3]:
dst_file_polygons = Path(dst_file_polygons)

if dst_file_polygons.exists():
    if not overwrite:
        raise FileExistsError(dst_file_polygons)
    else:
        print(f"Destination file will be overwritten: {dst_file_polygons}")


### Load input data

In [4]:
points = gpd.read_file(filename=src_file)
if src_keep_columns is not None:
    points = points[src_keep_columns]
points.head()

,id,tiles,geometry
0,Raedlinger Weiher,"32UQU, 32UQV, 33UUP, 33UUQ",POINT (12.57970 48.67412)
1,Lerchenauer See,32UPU,POINT (11.53678 48.19723)
2,Kieswerk Ebenhoeh,"32UPU, 32UQU",POINT (11.76433 48.18998)
3,Marzlinger Weiher,"32UPU, 32UQU",POINT (11.79569 48.39244)
4,Tegernsee,"32UPU, 32UQU, 32TPT, 32TQT",POINT (11.72314 47.74210)


### Prepare input data

In [5]:
assert "latitude" not in points.columns
assert "longitude" not in points.columns
assert "chip_id" not in points.columns

points["latitude"] = points.geometry.y
points["longitude"] = points.geometry.x
points = points.assign(chip_id=range(1, points.shape[0] + 1))
points.head()

,id,tiles,geometry,latitude,longitude,chip_id
0,Raedlinger Weiher,"32UQU, 32UQV, 33UUP, 33UUQ",POINT (12.57970 48.67412),48.674120,12.579700,1
1,Lerchenauer See,32UPU,POINT (11.53678 48.19723),48.197230,11.536780,2
2,Kieswerk Ebenhoeh,"32UPU, 32UQU",POINT (11.76433 48.18998),48.189980,11.764330,3
3,Marzlinger Weiher,"32UPU, 32UQU",POINT (11.79569 48.39244),48.392441,11.795693,4
4,Tegernsee,"32UPU, 32UQU, 32TPT, 32TQT",POINT (11.72314 47.74210),47.742100,11.723140,5


## Create chips

In [6]:
chips = geoimgchips.s2_raster_aligned_chips_from_points(points, buffer=buffer, res=res)
chips.head()

,id,tiles,geometry,latitude,longitude,chip_id,easting,northing,zone_number,zone_letter,epsg,minx,miny,maxx,maxy
1,Lerchenauer See,32UPU,"POLYGON ((11.51896 48.18607, 11.52010 48.20917...",48.197230,11.536780,2,688503.182448,5.341334e+06,32,U,32632,687220.0,5340050.0,689790.0,5342620.0
2,Kieswerk Ebenhoeh,"32UPU, 32UQU","POLYGON ((11.74638 48.17888, 11.74762 48.20198...",48.189980,11.764330,3,705439.586152,5.341111e+06,32,U,32632,704150.0,5339830.0,706720.0,5342400.0
3,Marzlinger Weiher,"32UPU, 32UQU","POLYGON ((11.77766 48.38130, 11.77892 48.40440...",48.392441,11.795693,4,706949.940679,5.363696e+06,32,U,32632,705660.0,5362410.0,708230.0,5364980.0
4,Tegernsee,"32UPU, 32UQU, 32TPT, 32TQT","POLYGON ((11.70539 47.73093, 11.70659 47.75403...",47.742100,11.723140,5,704137.085326,5.291228e+06,32,T,32632,702850.0,5289940.0,705420.0,5292510.0
0,Raedlinger Weiher,"32UQU, 32UQV, 33UUP, 33UUQ","POLYGON ((12.56278 48.66218, 12.56167 48.68528...",48.674120,12.579700,1,321827.842832,5.394057e+06,33,U,32633,320540.0,5392770.0,323110.0,5395340.0


## Save results

In [7]:
Path(dst_file_polygons).parent.mkdir(exist_ok=True)
chips.to_file(dst_file_polygons)